# ASBE - Automatic Stopping for Batch Experiments

> API details.

In [ ]:
#hide
from nbdev import *

In [ ]:
%nbdev_default_export core

Cells will be exported to asbe.core,
unless a different module is specified after an export flag: `%nbdev_export special.module`


In [ ]:
%nbdev_export
from modAL.models.base import BaseLearner
from sklearn.base import BaseEstimator, ClassifierMixin, RegressorMixin
from typing import Union, Optional
from copy import deepcopy
import numpy as np

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [ ]:
%nbdev_export
estimator_type = ClassifierMixin
class ASLearner(BaseLearner):
    def __init__(self,
                 estimator: Optional[estimator_type]=None, 
                 query_strategy=None,
                 assignment_fc=None
                ) -> None:
        self.estimator = estimator
        self.query_strategy = query_strategy
        self.assignment_fc = assignment_fc
        
    def teach(self, estimator.X_training, X_query):
        return(self.estimator.fit(np.hstack(estimator.X_training, X_query)))

SyntaxError: invalid syntax (<ipython-input-96-792a77490a53>, line 13)

In [ ]:
%nbdev_export
class ITEEstimator(BaseEstimator):
    """ Class for building a naive estimator for ITE estimation
    """
    def __init__(self,
                 model: estimator_type,
                 X_training,
                 t_training,
                 y_training,
                 X_test: np.ndarray = None,
                 two_model: bool = False,
                ) -> None:
        self.model = model
        self.X_training = X_training
        self.y_training = y_training
        self.t_training = t_training
        self.two_model  = two_model
        self.X_test = X_test
        self.N_training = X_training.shape[0]
    
    def fit(self):
        if self.two_model:
            self.m1 = deepcopy(self.model)
            control_ix = np.where(self.t_training == 0)[0]
            self.model.fit(self.X_training[control_ix,:],
                           self.y_training[control_ix])
            self.m1.fit(self.X_training[-control_ix,:],
                        self.y_training[-control_ix])
        else:
            self.model.fit(np.hstack((self.X_training,
                                      self.t_training.reshape((self.N_training, -1)))),
                           self.y_training)
            
    def predict(self, X=None):
        if X is None:
            X = self.X_test
        if self.two_model:
            self.y1_preds = self.m1.predict_proba(X)[:,1]
            self.y0_preds = self.model.predict_proba(X)[:,1]
        else:
            N_unlabeled = X.shape[0]
            self.y1_preds = self.model.predict_proba(
                                np.hstack((X,
                                np.ones(N_unlabeled).reshape(-1,1))))[:,1]
            self.y0_preds = self.model.predict_proba(
                np.hstack((X,
                           np.zeros(N_unlabeled).reshape(-1,1))))[:,1]
        return self.y1_preds - self.y0_preds, self.y1_preds, self.y0_preds

In [ ]:
X = np.random.normal(size = 1000).reshape((500,2))
t = np.random.binomial(n = 1, p = 0.5, size = 500)
y = np.random.binomial(n = 1, p = 1/(1+np.exp(X[:, 1]*2 + t*3)))
X_test = np.random.normal(size = 200).reshape((100,2))

In [ ]:
a = ITEEstimator(LogisticRegression(solver="lbfgs"), X, t, y, two_model = True)
# a = ITEEstimator(RandomForestClassifier(), X, t, y, X_test, two_model = False )

In [ ]:
a.fit()

In [ ]:
ite_pred, y1_pred, y0_pred = a.predict(X=a.X_training)

In [ ]:
np.mean(ite_pred)

-0.21278273360516106